In [1]:
%load_ext autoreload
%autoreload 2
import telepot
bot = telepot.Bot('436447719:AAGq_iK2hE3cPFhmL9Jh53g9HyVkw3SXZbg')
def qq(txt):
    bot.sendMessage(1114926, txt)

In [2]:
import faiss

ModuleNotFoundError: No module named 'faiss'

In [2]:
qq('start')

In [2]:
import json
import gzip
from itertools import islice
from collections import Counter, defaultdict
from operator import itemgetter
import pyarrow.parquet as pq
import pyarrow as pa

import pandas as pd
from tqdm.notebook import tqdm
import dill
import os
from pqdm.processes import pqdm
import re

from workers import *
    
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.callbacks import CallbackAny2Vec

import random
import collections
import gc

In [4]:
df = pd.read_pickle("df.pkl")

In [5]:
df.columns

Index(['vacancy_id', 'name', 'description', 'area_id', 'creation_date',
       'work_schedule', 'work_experience', 'compensation_from',
       'compensation_to', 'currency', 'key_skills', 'employer',
       'specializations', 'is_test', 'part', 'employment', 'spec_len',
       'groups_of_spec', 'len_groups', 'html_tags'],
      dtype='object')

In [6]:
df_test = df[df.is_test == True].copy()
del df

In [7]:
gc.collect()

20

In [8]:
df_test['clear_text'] = df_test.description.apply(remove_html)

In [9]:
qq('cleared')

In [15]:
df_test.to_parquet('df_test.pq')

In [3]:
df_test = pd.read_parquet('df_test.pq')

In [12]:
gc.collect()

20

In [13]:
%%time
if __name__ ==  '__main__': 
    df_test['processed_text'] = parallelize_on_rows(df_test.clear_text, preprocess_text) 

CPU times: user 10.8 s, sys: 16.3 s, total: 27.1 s
Wall time: 17min 3s


In [5]:
# %%time
# df_test['processed_text'] = df_test.clear_text.apply(preprocess_text) 

In [4]:
qq('processed')

In [17]:
df_text = df_test[['vacancy_id', 'name', 'specializations', 'key_skills', 'employer', 'groups_of_spec', 'processed_text']].copy()
del df_test
gc.collect()

595

In [18]:
df_text['key_skills'] = df_text['key_skills'].apply(' '.join)

In [22]:
df_text.to_parquet('test_text.pq')

In [5]:
df_text = pd.read_parquet('test_text.pq')

In [6]:
%%time
if __name__ ==  '__main__':
    df_text['key_skills'] = parallelize_on_rows(df_text.key_skills, preprocess_text)

CPU times: user 1.39 s, sys: 1.98 s, total: 3.37 s
Wall time: 34.8 s


In [7]:
%%time
if __name__ ==  '__main__':
    df_text['name'] = parallelize_on_rows(df_text.name, preprocess_text)

CPU times: user 1.47 s, sys: 1.46 s, total: 2.93 s
Wall time: 33 s


In [8]:
%%time
df_text['key_list'] = df_text['key_skills'].apply(lambda x: x.split(' '))
df_text['name_list'] = df_text['name'].apply(lambda x: x.split(' '))

CPU times: user 4.93 s, sys: 683 ms, total: 5.61 s
Wall time: 5.61 s


In [9]:
df_text.to_parquet('test_text.pq')

In [10]:
%%time
df_text['text_list'] = df_text['processed_text'].apply(lambda x: x.split(' '))

CPU times: user 17.2 s, sys: 5.74 s, total: 23 s
Wall time: 23 s


In [16]:
df_text.drop(columns = ['text_list'], inplace = True)

In [10]:
model = Doc2Vec.load('models/vector_size:300_min_count:2_epochs:10_window:7_seed:42_workers:4_negative:50.bin')

In [12]:
def inf_vec(txt):
    return model.infer_vector(txt.split(' '))

In [13]:
df_text['text_vector'] = pqdm(df_text['processed_text'], inf_vec, n_jobs=10)

In [18]:
df_text.head()

vacancy_id                                     name specializations  \
0     2700001                     прораб великий устюг            None   
1     2700002  лаборант электротехнический лаборатория            None   
2     2700003         продавец консультант радуга парк            None   
3     2700004      тестировщик команда saving products            None   
7     2700009          сварщик аргонщик ручной дуговой            None   

                                          key_skills      employer  \
0                                                     72dc8d78bbec   
1  office пользователь деловой общение деловой пе...  ba6dcbd0bcd1   
2  грамотный речь коммуникабельность активный про...  ddc67b017661   
3                                                     a621c7f876ec   
7                                   сварочный работа  d4e25650ee12   

  groups_of_spec                                     processed_text  \
0           None  обязанность взаимодействие субподрядчик предос...   
1           None  связь расширение компания требоваться quot лаб...   
2           None  обязанность консультирование клиент торговый з...   
3           None  команда saving products обеспечивать оформлени...   
7           None  обязанность работа нержавейка толщина стенка д...   

                                            key_list  \
0                                                 []   
1  [office, пользователь, деловой, общение, делов...   
2  [грамотный, речь, коммуникабельность, активный...   
3                                                 []   
7                                [сварочный, работа]   

                                     name_list  \
0                     [прораб, великий, устюг]   
1  [лаборант, электротехнический, лаборатория]   
2        [продавец, консультант, радуга, парк]   
3     [тестировщик, команда, saving, products]   
7         [сварщик, аргонщик, ручной, дуговой]   

                                         text_vector  
0  [-0.050694242, 0.4064839, 0.55667454, -0.16124...  
1  [-0.19709699, -0.35366777, 0.10756227, 0.23191...  
2  [-0.19167374, 0.19502771, 0.16828729, -0.04970...  
3  [-0.7373982, -0.07595682, 1.0881921, -0.209517...  
7  [-0.27458248, -0.07573847, 0.30837265, 0.08945...

In [17]:
df_text.to_parquet('df_vector_test.pq')

In [23]:
res = model.docvecs.most_similar([df_text.loc[0].text_vector], topn=10)

In [33]:
def get_similar(vect):
    return model.docvecs.most_similar([vect], topn=3)

In [3]:
df_text= pd.read_parquet('df_vector_test.pq')

In [31]:
%%time
df_text.iloc[:1000]['text_vector'].apply(get_similar)

CPU times: user 7min 57s, sys: 4.36 s, total: 8min 1s
Wall time: 1min 20s


0       [(1295984, 0.7789467573165894)]
1       [(2524055, 0.8494123220443726)]
2        [(935704, 0.8325062990188599)]
3       [(1714067, 0.7315236330032349)]
7       [(2426502, 0.8062281012535095)]
                     ...               
2062    [(1051744, 0.8391262292861938)]
2063    [(2686562, 0.8080798387527466)]
2064    [(2532126, 0.7068248391151428)]
2069    [(1756433, 0.6649709343910217)]
2071     [(311097, 0.7944725155830383)]
Name: text_vector, Length: 1000, dtype: object

In [32]:
df_text.shape[0]/1000*80/3600

32.36277777777778

In [ ]:
%%time
df_text['similar']=df_text['text_vector'].apply(get_similar)

In [ ]:
qq('similarity - done')

In [25]:
vac_id = [id for id, sim in res]

In [26]:
df_tarin = pd.read_parquet('df_tarin.pq')

In [35]:
print(df_text.iloc[0].processed_text)

обязанность взаимодействие субподрядчик предоставление фронт работа контроль качество смр срок соответствие утверждать график контроль качество материал использовать работа подрядчик учет выполнять работа субподрядный организация участие подготовка перечень замечание выполнять работа субподрядный организация контроль исполнение замечание дефектный ведомость сбор подрядчик исполнительный документация составление план работа субподрядный организация выявление фактор препятствовать выполнение работа информирование руководитель контроль срок выполнение работа прямой подрядчик застройщик совмещать график работа планирование работа стэп своевременный выявление потребность ресурс участок формирование заявка ресурс выполнение работа сила стэп учет ресурс предоставление первичный отчетность выполнение работа сила стэп контроль исполнение правило инструктаж вверять рабочий начало работа осмотр место производство работа безопасность подтверждение объем работа выполнять субподрядчик подготовка баз

In [ ]:
print(df_tarin[df_tarin.vacancy_id == vac_id[1]].clear_text.values)